In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv


In [172]:
# Reading Data in the dataframe
df_initial = pd.read_csv(r'/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')

df_initial.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [173]:
# Columns in dataset df_initial
df_initial.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [174]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score


In [175]:
# Creating df with selected/important columns
df = df_initial[['title', 'location', 'company_profile', 'description', 'requirements', 'industry', 'fraudulent']]
# Dropping rows having NaN values
df_new = df.dropna()

# Columns to retain
features = ['title', 'location', 'company_profile', 'description', 'requirements', 'industry']
target = ['fraudulent']

X = df_new[features]
y = df_new['fraudulent']

# Spliting data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 1)

In [176]:
# Defining CounterVectorizer

count_vect = CountVectorizer(min_df=3, ngram_range=[1, 2]).fit(X_train['title']+' '+X_train['location']+' '+X_train['company_profile']+' '+X_train['description']+' '+X_train['requirements']+' '+X_train['industry'])
X_train_count_vect = count_vect.transform(X_train['title']+' '+X_train['location']+' '+X_train['company_profile']+' '+X_train['description']+' '+X_train['requirements']+' '+X_train['industry'])
X_test_count_vect = count_vect.transform(X_test['title']+' '+X_test['location']+' '+X_test['company_profile']+' '+X_test['description']+' '+X_test['requirements']+' '+X_test['industry'])


In [177]:
# Defining TfidfVectorizer

tfidf_vect = TfidfVectorizer(min_df=3, ngram_range=[1, 2]).fit(X_train['title']+' '+X_train['location']+' '+X_train['company_profile']+' '+X_train['description']+' '+X_train['requirements']+' '+X_train['industry'])
X_train_tfidf_vect = tfidf_vect.transform(X_train['title']+' '+X_train['location']+' '+X_train['company_profile']+' '+X_train['description']+' '+X_train['requirements']+' '+X_train['industry'])
X_test_tfidf_vect = tfidf_vect.transform(X_test['title']+' '+X_test['location']+' '+X_test['company_profile']+' '+X_test['description']+' '+X_test['requirements']+' '+X_test['industry'])


In [178]:
# Using LogisticRegression and CounterVectorizer for calculating roc_auc_score

lr_cv_model = LogisticRegression(C=0.1, max_iter=500, random_state = 1).fit(X_train_count_vect, y_train)
y_pred_lr_cv = lr_cv_model.predict(X_test_count_vect)
score_lr_cv = roc_auc_score(y_test, y_pred_lr_cv)
print("roc_auc_score using LogisticRegression and CounterVectorizer: ", score_lr_cv)

# get the feature names as numpy array
feature_names = np.array(count_vect.get_feature_names())
# Sort the coefficients from the model
sorted_coef_index = lr_cv_model.coef_[0].argsort()
# Find the 10 smallest and 10 largest coefficients
print('\n\nSmallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

roc_auc_score using LogisticRegression and CounterVectorizer:  0.96


Smallest Coefs:
['we' 'is' 'based' 'it' 'are' 'an' 'you' 'staff' 'recruitment' 'for']

Largest Coefs: 
['hotel' 'financing' 'engineering' 'controls' 'industry' 'accion' 'more'
 'money' 'is to' 'tx']


In [179]:
# Using LogisticRegression and TfidfVectorizer for calculating roc_auc_score

lr_tfidf_model = LogisticRegression(C=15, max_iter=500, random_state = 1).fit(X_train_tfidf_vect, y_train)
y_pred_lr_tfidf = lr_tfidf_model.predict(X_test_tfidf_vect)
score_lr_tfidf = roc_auc_score(y_test, y_pred_lr_tfidf)
print("roc_auc_score using LogisticRegression and TfidfVectorizer: ", score_lr_tfidf)


# get the feature names as numpy array
feature_names = np.array(tfidf_vect.get_feature_names())
# Sort the coefficients from the model
sorted_coef_index = lr_tfidf_model.coef_[0].argsort()
# Find the 10 smallest and 10 largest coefficients
print('\n\nSmallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


roc_auc_score using LogisticRegression and TfidfVectorizer:  0.94


Smallest Coefs:
['you' 'we' 'of' 'it' 'social' 're' 'is' 'are' 'be' 'web']

Largest Coefs: 
['hotel' 'accion' 'engineering' 'controls' 'and' 'payroll' 'novation'
 'aptitude staffing' 'financing' 'american secured']


In [180]:
# Using MultinomialNB and CounterVectorizer for calculating roc_auc_score

mnb_cv_model = MultinomialNB(alpha=0.1).fit(X_train_count_vect, y_train)
y_pred_mnb_cv = mnb_cv_model.predict(X_test_count_vect)
score_mnb_cv = roc_auc_score(y_test, y_pred_mnb_cv)
print("roc_auc_score using MultinomialNB and CounterVectorizer: ", score_mnb_cv)

# get the feature names as numpy array
feature_names = np.array(count_vect.get_feature_names())
# Sort the coefficients from the model
sorted_coef_index = mnb_cv_model.coef_[0].argsort()
# Find the 10 smallest and 10 largest coefficients
print('\n\nSmallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

roc_auc_score using MultinomialNB and CounterVectorizer:  0.97


Smallest Coefs:
['looking for aurora' 'policies perform' 'policies on'
 'policies objectives initiatives' 'policies objectives'
 'policies load unload' 'policies load' 'policies including health'
 'policies practices' 'policies including']

Largest Coefs: 
['and' 'to' 'the' 'of' 'in' 'with' 'for' 'our' 'is' 'experience']


In [181]:
# Using MultinomialNB and TfidfVectorizer for calculating roc_auc_score

mnb_tfidf_model = MultinomialNB(alpha=0.01).fit(X_train_tfidf_vect, y_train)
y_pred_mnb_tfidf = mnb_tfidf_model.predict(X_test_tfidf_vect)
score_mnb_tfidf = roc_auc_score(y_test, y_pred_mnb_tfidf)
print("roc_auc_score using MultinomialNB and TfidfVectorizer: ", score_mnb_tfidf)


# get the feature names as numpy array
feature_names = np.array(tfidf_vect.get_feature_names())
# Sort the coefficients from the model
sorted_coef_index = mnb_tfidf_model.coef_[0].argsort()
# Find the 10 smallest and 10 largest coefficients
print('\n\nSmallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


roc_auc_score using MultinomialNB and TfidfVectorizer:  0.97


Smallest Coefs:
['looking for aurora' 'policies perform' 'policies on'
 'policies objectives initiatives' 'policies objectives'
 'policies load unload' 'policies load' 'policies including health'
 'policies practices' 'policies including']

Largest Coefs: 
['and' 'to' 'the' 'of' 'in' 'with' 'for' 'oil' 'our' 'engineering']
